# Regional Base Ranking Algorithm

The recommendation is rank trails based on 2 components: region and ratings.

If the hiking trail is within user's location region, then those trails will be moved to the top of the list.

Then, the algorithm will then check out the regions that are next to the user's region, which the regions will be sorted based on their average rating.

- Note: the average rating is the calculated by the sum of all the trail's rating over the total number of trails based on their region.





In [1]:
# Import everything that we need
from dataset import dataSet # Access the sample dataSet
from random import randint

# Create our Graph

In order to get the neighbour regions from the user's region, we'll first have to create a graph.

This is the map of BC

- Look closely of how each region are connected to each other.

![mapOfBC](imgSrc/MapBC.png)

Then, we'll create the graph based on the map, so we can now see the neighbour of each regions.

![graphNeighbour](imgSrc/GraphNeighbourRegion.png)

In [2]:
# Declare the graph
regionGraph = {
    "Pemberton": ["Whistler"],
    "Whistler": ["Pemberton", "Howe Sound"],
    "Howe Sound": ["Whistler", "Sunshine Coast", "The North Shore"],
    "Sunshine Coast": ["Howe Sound"],
    "The North Shore": ["Howe Sound", "Tri Cities", "Vancouver City"],
    "Vancouver City": ["The North Shore", "Surrey and Langley", "Tsawwassen and Delta", "Tri Cities"],
    "Fraser Valley East": ["Ridge Meadows", "Surrey and Langley", "Manning Provincial Park"],
    "Tri Cities": ["The North Shore", "Surrey and Langley", "Ridge Meadows", "Vancouver City"],
    "Tsawwassen and Delta": ["Vancouver City", "Surrey and Langley"],
    "Surrey and Langley": ["Tri Cities", "Vancouver City", "Tsawwassen and Delta", "Fraser Valley East", "Ridge Meadows"],
    "Ridge Meadows": ["Tri Cities", "Surrey and Langley", "Fraser Valley East"],
    "Manning Provincial Park": ["Fraser Valley East"]
}

# Helper function

* Create function that will get average rate of the region
    - **Note:** the average rating is the calculated by the sum of all the trail's rating over the total number of trails based on their region.

* Create function that will sort neighbours based on their average ratings
    - Using bubble sort to sort neighbour regions based on their average rattings.

* Create function that will sort those hiking trails based on their ratings
    - Applying quicksort to sort those trails in descending order (ratings)

In [3]:
# Get average rate of the region
def getAvgRate(region:str):
    totalTrail = 0
    totalRate = 0
    for trail in dataSet:
        if (trail['region'] == region):
            totalRate += trail['rate']
            totalTrail += 1
    
    return round(totalRate / totalTrail, 2)


# Sort neighbours based on their average ratings
def sortNeighbours(arr:list):
    n = len(arr) 
    if n == 1:
        return arr
    else:
        for i in range(n):
            for j in range(n-i-1):
                if (getAvgRate(arr[j]) < getAvgRate(arr[j+1])):
                    tmp = arr[j]
                    arr[j] = arr[j+1]
                    arr[j+1] = tmp
    return arr


# Get list of trails based on their region
def getRegionList(region: str):
    res = []
    for trail in dataSet:
        if (trail["region"] == region):
            res.append(trail)
    return res


# Get list of trails based on their region in decending order (based on their ratings)
def sortRegionList(dataSet: list):
    if (len(dataSet) <= 1):
        return dataSet

    smaller = []
    equal = []
    larger = []
    pivot = dataSet[randint(0, len(dataSet)-1)]["rate"]

    for trail in dataSet:
        if (trail["rate"] < pivot):
            smaller.append(trail)
        elif (trail["rate"] == pivot):
            equal.append(trail)
        else:
            larger.append(trail)
    
    larger = sortRegionList(larger)
    smaller = sortRegionList(smaller)

    return larger + equal + smaller

# The Algorithm

- First put those trails that are within user's region first

- Then add the other trails from the neighbour regions (based on the region's average ratings)

- Then add the rest in the end

In [4]:
# The Trail recommendation algorithm
def trailRecommendation(region: str):
    res = []
    tmpRegions = []

    # Add the user's region
    currRegionArr = sortRegionList(getRegionList(region))
    res += currRegionArr
    tmpRegions.append(region)
    neighbourRegions = sortNeighbours(regionGraph[region])

    # Adding neignbour regions
    for region in neighbourRegions:
        tmpRegions.append(region)
        res += sortRegionList(getRegionList(region))

    # Add left over regions
    for region in regionGraph:
        if (region not in tmpRegions):
            tmpRegions.append(region)
            res += sortRegionList(getRegionList(region))

    return res


In [6]:
# Test out the function
def printJSON(jArr):
    for i in jArr:
        print(f"id: {i['id']},\nregion: {i['region']},\nTrail Title: {i['trailTitle']},\nDifficulty: {i['difficulty']},\nTime: {i['time']},\nRating: {i['rate']},\nTrip Time: {i['tripTime']},\nElevation Gain: {i['elevationGain']},\nSeason: {i['season']},\nCamping: {i['camping']},\nPublic Transit: {i['publicTransit']},\nDog Friendly: {i['dogFriendly']}\n\n")
printJSON(trailRecommendation('Vancouver City'))

id: 37a8ccec053211ec9f58cf7d0294de9c,
region: Vancouver City,
Trail Title: Deep lake,
Difficulty: Easy,
Time: 1,
Rating: 4.6,
Trip Time: 5,
Elevation Gain: 152,
Season: year-round,
Camping: False,
Public Transit: True,
Dog Friendly: True


id: 4b4e4738052611ec9f58cf7d0294de9c,
region: Vancouver City,
Trail Title: Burnaby Mountain,
Difficulty: Easy,
Time: 3,
Rating: 3.83,
Trip Time: 8,
Elevation Gain: 300,
Season: year-round,
Camping: False,
Public Transit: True,
Dog Friendly: True


id: fbda5a40057311ec9f58cf7d0294de9c,
region: Vancouver City,
Trail Title: Foreshore Trail,
Difficulty: Easy,
Time: 3,
Rating: 3.7,
Trip Time: 5,
Elevation Gain: 40,
Season: year-round,
Camping: False,
Public Transit: True,
Dog Friendly: False


id: 4b478394052611ec9f58cf7d0294de9c,
region: Vancouver City,
Trail Title: Burnaby Lake,
Difficulty: Easy,
Time: 2,
Rating: 3.27,
Trip Time: 10,
Elevation Gain: 152,
Season: year-round,
Camping: False,
Public Transit: True,
Dog Friendly: True


id: 3730fb9a053211ec9